In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import f1_score
import nltk
import ssl
from nltk.corpus import stopwords
import re
import pymorphy2

In [3]:
%%time
#читаем csv
#%%time
df = pd.read_csv('C:\Temp\lenta-ru-news.csv')

<timed exec>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 1min 25s


In [4]:
# в новый dateframe записываем только нужные нам темы
df_2 = df[df.topic.isin(['Россия','Мир','Экономика','Спорт','Культура'])].copy()
df_2= df_2.drop(columns = ['url', 'tags','date'], axis = 1)

In [5]:
RANDOM_STATE = 123

In [6]:
df_2.shape

(494804, 3)

In [7]:
df_2.topic.value_counts()

Россия       160445
Мир          136621
Экономика     79528
Спорт         64413
Культура      53797
Name: topic, dtype: int64

In [8]:
df_2.isna().sum()

title    0
text     5
topic    0
dtype: int64

In [9]:
#удаляем строки без текста
df_2.dropna(subset=['text'],inplace = True)

In [10]:
df_2.isna().sum()

title    0
text     0
topic    0
dtype: int64

In [11]:
df_2.head()

,title,text,topic
5,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия
6,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия
7,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия
8,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир
9,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия


In [ ]:
#работаем с текстом

In [11]:
text_transformer = TfidfVectorizer()

In [12]:
%%time
text = text_transformer.fit_transform(df_2['text'])

Wall time: 1min 56s


In [15]:
text.shape

(494799, 821732)

In [18]:
%%time
X_train, X_test, y_train, y_test = train_test_split(text, df_2['topic'], test_size=0.20, random_state=RANDOM_STATE)

Wall time: 605 ms


In [22]:
%%time
clf = tree.DecisionTreeClassifier()

Wall time: 0 ns


In [23]:
#очень долгие выисления, на моей тестовой машине 2 часа 14 минут
%%time
clf.fit(X_train,y_train)

Wall time: 2h 14min 11s


DecisionTreeClassifier()

In [38]:
%%time
pred = clf.predict(X_test)

Wall time: 478 ms


In [40]:
pred

array(['Мир', 'Мир', 'Культура', ..., 'Россия', 'Мир', 'Россия'],
      dtype=object)

In [35]:
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

Качество модели по метрике F1 0.7916565805994169


In [ ]:
#преобразуем текст

In [45]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tolyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
stop_russian = stopwords.words('russian')

In [13]:
text_transformer2 = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True)

In [14]:
# #Что я только не делал, не работает в итоге похоже просто перезапуск всего и вся помог

# # функция удаления пунктуации и цифр
# def remove_trash(str): 
#     pattern = r'[^А-Яа-я]+'
#     try:
#         #не знаю почему не работала ваша функция, пришлось добавить join чтобы оно собиралось в строку
#       str = ''.join([re.sub(pattern, ' ', i) for i in str])
# #       print (list)
#     except Exception as e:
#       print(e)
#     return str

def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
#       print (list)
    except Exception as e:
      print(e)
    return list

In [14]:
#df_3=df_2.copy()

In [15]:
%%time
df_2['text_clean'] = remove_trash(df_2['text'])

Wall time: 41.4 s


In [16]:
df_2['text'][23]

'"Женщины России" , возмущенные отсутствием своих кандидаток в региональных кандидатских списках, вышли из блока "Отечество - Вся Россия" Лидер Движения в поддержку армии (ДПА) Виктор Илюхин заверил агентство "Интерфакс" в том, что не собирается входить со своим движением в избирательный блок "За победу", создаваемый КПРФ. Сегодня Илюхин заявил, что ДПА идет на выборы самостоятельно. Он, Илюхин не одобряет тактики коммунистов и считает свое движение самодостаточным, раз коммунисты сами заявили о своей самостоятельности еще в 1998 году. Илюхин не принял предложения патриотического объединения "Русский дом" возглавить новый блок, а предупредил, что установит для будущих попутчиков из других движений жесткие квоты на кандидатские места. Илюхин уверен, что преодолеет 5-процентный барьер. Тем временем Владимир Исаков, когда-то один из первых оппонентов президента Ельцина на съезде народных депутатов РФ, а ныне руководитель правового управления Госдумы, объяснил корреспонденту Интерфакса неу

In [17]:
df_2['text_clean'][23]

' Женщины России возмущенные отсутствием своих кандидаток в региональных кандидатских списках вышли из блока Отечество Вся Россия Лидер Движения в поддержку армии ДПА Виктор Илюхин заверил агентство Интерфакс в том что не собирается входить со своим движением в избирательный блок За победу создаваемый КПРФ Сегодня Илюхин заявил что ДПА идет на выборы самостоятельно Он Илюхин не одобряет тактики коммунистов и считает свое движение самодостаточным раз коммунисты сами заявили о своей самостоятельности еще в году Илюхин не принял предложения патриотического объединения Русский дом возглавить новый блок а предупредил что установит для будущих попутчиков из других движений жесткие квоты на кандидатские места Илюхин уверен что преодолеет процентный барьер Тем временем Владимир Исаков когда то один из первых оппонентов президента Ельцина на съезде народных депутатов РФ а ныне руководитель правового управления Госдумы объяснил корреспонденту Интерфакса неудачу КПРФ с созданием блока За победу п

In [18]:
morph = pymorphy2.MorphAnalyzer()

In [19]:
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [20]:
#df_4=df_3.copy()
#df_4.head()

,title,text,topic,text_clean
5,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Как стало известно агентству Ассошиэйтед Пресс...
6,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,В зале игровых автоматов в третьем ярусе подзе...
7,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Япония приняла решение разморозить кредиты Рос...
8,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Британцы отмечают сегодня скорбную дату вторую...
9,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,В понедельник директор департамента внешних св...


In [20]:
%%time
#после увеличения ОЗУ прошло 1 час быстрее,  Wall time: 5h 52min 1s , а было в первой попытке почти 7 часов 
df_2['text_clean_normal'] = df_2.apply(lemmatize,axis=1)

Wall time: 5h 52min 1s


In [21]:
%%time
text_norm = text_transformer2.fit_transform(df_2['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df_2['topic'], test_size=0.20, random_state=RANDOM_STATE)

Wall time: 1min 27s


In [22]:
%%time
#Wall time: 1h 9min 55s видно что толи нормализация текста, толи увеличение ОЗУ с 6 до 12 
#обучение позволило провести в два раза быстрее
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)


Wall time: 1h 9min 55s


DecisionTreeClassifier()

In [23]:
%%time
pred_norm = clf_norm.predict(X_test)

Wall time: 630 ms


In [24]:
pred_norm

array(['Мир', 'Мир', 'Культура', ..., 'Россия', 'Мир', 'Россия'],
      dtype=object)

In [25]:
print('Качество модели по метрике F1 после нормализации текста', f1_score(y_test,pred_norm,average='weighted'))

Качество модели по метрике F1 после нормализации текста 0.8178625612235791


Удивило что качество модели по метрике F1 0.7916565805994169 до нормализации, а после всего 0.8178625612235791 по одной и той же модели  классификации 
то есть вcе наши действия в несколько часов обработки увечили качество всего на 2,5 процента. 
но в целом было интересно. посмотреть как измениться текст